In [1]:
import requests
import json
from pprint import pprint
import os
from pathlib import Path

AV_KEY = "YD9VPN2U7FKOT5VT"

# Cache configuration
CACHE_DIR = Path("./av_cache")
CACHE_DIR.mkdir(exist_ok=True)

def _get_cache_file(symbol: str, data_type: str) -> Path:
    """Get the cache file path for a symbol and data type."""
    return CACHE_DIR / f"{symbol}_{data_type}.json"

def _load_from_cache(symbol: str, data_type: str) -> dict:
    """Load data from cache if it exists."""
    cache_file = _get_cache_file(symbol, data_type)
    if cache_file.exists():
        print(f"Loading {data_type} from cache for {symbol}...")
        with open(cache_file, 'r') as f:
            return json.load(f)
    return None

def _save_to_cache(symbol: str, data_type: str, data: dict):
    """Save data to cache."""
    cache_file = _get_cache_file(symbol, data_type)
    with open(cache_file, 'w') as f:
        json.dump(data, f, indent=2)
    print(f"Saved {data_type} to cache for {symbol}")

def fetch_company_data(symbol: str, use_cache: bool = True) -> dict:
    """Fetch company overview, income statement, balance sheet, and cash flow statements.
    
    Args:
        symbol: Stock ticker symbol (e.g., 'CROX')
        use_cache: Whether to use cached data if available
    
    Returns:
        dict with keys: overview, income_statement, balance_sheet, cash_flow
    """
    symbol = symbol.upper()
    result = {}
    
    # Define API functions to fetch
    functions = {
        "OVERVIEW": "overview",
        "INCOME_STATEMENT": "income_statement",
        "BALANCE_SHEET": "balance_sheet",
        "CASH_FLOW": "cash_flow"
    }
    
    for func, data_type in functions.items():
        # Try to load from cache first
        if use_cache:
            cached_data = _load_from_cache(symbol, data_type)
            if cached_data is not None:
                result[data_type] = cached_data
                continue
        
        # Fetch from API
        print(f"Fetching {data_type} from API for {symbol}...")
        url = f"https://www.alphavantage.co/query?function={func}&symbol={symbol}&apikey={AV_KEY}"
        
        response = requests.get(url)
        data = response.json()
        
        # Save to cache
        _save_to_cache(symbol, data_type, data)
        result[data_type] = data
    
    return result

# Fetch data for CROX
symbol = "CMG"
company_data = fetch_company_data(symbol, use_cache=True)

# Display the collected data
print("\n" + "=" * 60)
print(f"Company Data for {symbol}")
print("=" * 60)

print("\n--- COMPANY OVERVIEW ---")
pprint(company_data["overview"])

print("\n--- INCOME STATEMENT ---")
pprint(company_data["income_statement"].keys())  # Show available keys

print("\n--- BALANCE SHEET ---")
pprint(company_data["balance_sheet"].keys())  # Show available keys

print("\n--- CASH FLOW STATEMENT ---")
pprint(company_data["cash_flow"].keys())  # Show available keys

Loading overview from cache for CMG...
Loading income_statement from cache for CMG...
Loading balance_sheet from cache for CMG...
Loading cash_flow from cache for CMG...

Company Data for CMG

--- COMPANY OVERVIEW ---
{'200DayMovingAverage': 47.34,
 '50DayMovingAverage': 38.36,
 '52WeekHigh': 66.74,
 '52WeekLow': 29.75,
 'Address': '610 NEWPORT CENTER DRIVE, NEWPORT BEACH, CA, UNITED STATES, 92660',
 'AnalystRatingBuy': 21,
 'AnalystRatingHold': 10,
 'AnalystRatingSell': 0,
 'AnalystRatingStrongBuy': 5,
 'AnalystRatingStrongSell': 0,
 'AnalystTargetPrice': 44.09,
 'AssetType': 'Common Stock',
 'Beta': 0.999,
 'BookValue': 2.43,
 'CIK': '1058090',
 'Country': 'USA',
 'Currency': 'USD',
 'Description': 'Chipotle Mexican Grill, Inc. is a prominent fast-casual '
                'restaurant chain celebrated for its dedication to '
                'high-quality, customizable Mexican-inspired cuisine, anchored '
                'in fresh and sustainably sourced ingredients. With a robust '
  

In [2]:
# Build TTM (trailing twelve months) statements from the latest 4 quarterly reports
# The code below expects `company_data` (from the previous cell) or will load cached files if not present.
from pathlib import Path

CACHE_DIR = Path("./av_cache")

def _convert_num(s):
    """Convert a string numeric value to int or float. Returns None if empty or not numeric."""
    if s is None:
        return None
    s = str(s).strip()
    if s == "" or s.lower() == "none":
        return None
    # remove commas
    s = s.replace(",", "")
    # handle values like "(123)" if any (treat as negative)
    if s.startswith("(") and s.endswith(")"):
        s = "-" + s[1:-1]
    try:
        # prefer integer when there is no decimal point
        if "." in s:
            return float(s)
        return int(s)
    except Exception:
        try:
            return float(s)
        except Exception:
            return None


def build_ttm_from_statement(stmt: dict):
    """Construct a TTM statement by summing the latest 4 quarterlyReports.

    Returns a dict with the same keys as the quarterly reports. The
    'fiscalDateEnding' and 'reportedCurrency' are taken from the latest
    quarter. Numeric fields are converted and summed.
    """
    if not stmt:
        return None
    qreports = stmt.get("quarterlyReports") or []
    if len(qreports) == 0:
        return None
    latest4 = qreports[:4]

    # collect all keys present in these quarters
    keys = set()
    for d in latest4:
        keys.update(d.keys())

    ttm = {}
    for k in keys:
        if k in ("fiscalDateEnding", "reportedCurrency"):
            ttm[k] = latest4[0].get(k)
            continue
        # convert each quarter's value
        vals = [_convert_num(d.get(k)) for d in latest4]
        # if all values are None -> keep None
        if all(v is None for v in vals):
            ttm[k] = None
            continue
        # otherwise sum treating missing as 0
        total = 0
        any_float = False
        for v in vals:
            if v is None:
                continue
            if isinstance(v, float):
                any_float = True
            total += v
        # if any value was float, keep float; else convert to int
        if any_float:
            ttm[k] = float(total)
        else:
            try:
                ttm[k] = int(total)
            except Exception:
                ttm[k] = total

    # attach metadata about which quarters were used
    ttm["_ttm_from_quarters"] = [d.get("fiscalDateEnding") for d in latest4]
    return ttm


# Ensure we have company_data available (it was created in the previous cell); if not, try to load from cache
try:
    company_data  # noqa: F821
except NameError:
    print("company_data not found in the notebook environment, loading from cache for CROX...")
    symbol = "CROX"
    def _load_cache(symbol, data_type):
        p = CACHE_DIR / f"{symbol}_{data_type}.json"
        if p.exists():
            return json.loads(p.read_text(encoding='utf-8'))
        return None
    company_data = {
        "overview": _load_cache(symbol, "overview"),
        "income_statement": _load_cache(symbol, "income_statement"),
        "balance_sheet": _load_cache(symbol, "balance_sheet"),
        "cash_flow": _load_cache(symbol, "cash_flow"),
    }

# Build TTM statements
income_ttm = build_ttm_from_statement(company_data.get("income_statement"))
balance_ttm = build_ttm_from_statement(company_data.get("balance_sheet"))
cashflow_ttm = build_ttm_from_statement(company_data.get("cash_flow"))

print("\n--- TTM INCOME STATEMENT ---")
if income_ttm:
    pprint(income_ttm)
else:
    print("No income statement data available")

print("\n--- TTM BALANCE SHEET ---")
if balance_ttm:
    pprint(balance_ttm)
else:
    print("No balance sheet data available")

print("\n--- TTM CASH FLOW STATEMENT ---")
if cashflow_ttm:
    pprint(cashflow_ttm)
else:
    print("No cash flow data available")



--- TTM INCOME STATEMENT ---
{'_ttm_from_quarters': ['2025-09-30', '2025-06-30', '2025-03-31', '2024-12-31'],
 'comprehensiveIncomeNetOfTax': None,
 'costOfRevenue': 8755704000,
 'costofGoodsAndServicesSold': 8755704000,
 'depreciation': None,
 'depreciationAndAmortization': 352556000,
 'ebit': 1997170000,
 'ebitda': 2349726000,
 'fiscalDateEnding': '2025-09-30',
 'grossProfit': 3031696000,
 'incomeBeforeTax': 2014973000,
 'incomeTaxExpense': 478380000,
 'interestAndDebtExpense': None,
 'interestExpense': None,
 'interestIncome': 83762000,
 'investmentIncomeNet': None,
 'netIncome': 1536593000,
 'netIncomeFromContinuingOperations': 1536593000,
 'netInterestIncome': 83762000,
 'nonInterestIncome': None,
 'operatingExpenses': 1100485000,
 'operatingIncome': 1931211000,
 'otherNonOperatingIncome': None,
 'reportedCurrency': 'USD',
 'researchAndDevelopment': None,
 'sellingGeneralAndAdministrative': 682892000,
 'totalRevenue': 11787400000}

--- TTM BALANCE SHEET ---
{'_ttm_from_quarters':